In [1]:
import pandas as pd
import numpy as np
import re

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)  # me muestre todas las columnas
#pd.set_option('display.max_rows', None)  # me muestre todas las filas

# pa pintar
import pylab as plt   # import matplotlib.pyplot as plt
import seaborn as sns

# para que salga el grafico
%matplotlib inline

In [2]:
df_origin = pd.read_csv('attacks.csv', encoding='latin1')
df = df_origin.copy()

In [3]:
df.head(10)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN
5,2018.06.03.b,03-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.b,2018.06.03.b,6298.0,NaN,NaN
6,2018.06.03.a,03-Jun-2018,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.03.a,2018.06.03.a,6297.0,NaN,NaN
7,2018.05.27,27-May-2018,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.27,2018.05.27,6296.0,NaN,NaN
8,2018.05.26.b,26-May-2018,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.b,2018.05.26.b,6295.0,NaN,NaN
9,2018.05.26.a,26-May-2018,2018.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",Standing,male,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.05.26.a,2018.05.26.a,6294.0,NaN,NaN


In [4]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

In [6]:
df.shape

(25723, 24)

In [7]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

Case Number               17021
Date                      19421
Year                      19423
Type                      19425
Country                   19471
Area                      19876
Location                  19961
Activity                  19965
Name                      19631
Sex                       19986
Age                       22252
Injury                    19449
Fatal (Y/N)               19960
Time                      22775
Species                   22259
Investigator or Source    19438
pdf                       19421
href formula              19422
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Unnamed: 22               25722
Unnamed: 23               25721
dtype: int64

In [8]:
df.duplicated().any()

True

In [9]:
df = df.drop_duplicates()

In [10]:
df.shape, df_origin.shape

((6312, 24), (25723, 24))

In [11]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


### Número de nulos por cada columna

In [12]:
nan_cols = df.isna().sum()

nan_cols[nan_cols>0]

Case Number                  2
Date                        10
Year                        12
Type                        14
Country                     60
Area                       465
Location                   550
Activity                   554
Name                       220
Sex                        575
Age                       2841
Injury                      38
Fatal (Y/N)                549
Time                      3364
Species                   2848
Investigator or Source      27
pdf                         10
href formula                11
href                        10
Case Number.1               10
Case Number.2               10
original order               3
Unnamed: 22               6311
Unnamed: 23               6310
dtype: int64

In [13]:
df.reset_index(drop=True, inplace=True)

In [14]:
nan_rows = df.isna().sum(axis=1)

# Filtrar las filas con valores nulos
rows_with_nulls = nan_rows[nan_rows > 0]

rows_with_nulls.tail()

6307    22
6308    22
6309    23
6310    24
6311    23
dtype: int64

# 1. Limpieza de las columnas (Case Number, Date,	Year, Case Number.1,	Case Number.2,	original order,	Unnamed: 22,	Unnamed: 23	)

### Posición de los valores nulos en 'Date'

In [15]:
bad_index = df['Date'][df['Date'].isna()].index

bad_index

Int64Index([6302, 6303, 6304, 6305, 6306, 6307, 6308, 6309, 6310, 6311], dtype='int64')

In [16]:
# Obtener filas por índice o etiqueta de fila
df.loc[[6302, 6303, 6304, 6305, 6306, 6307, 6308, 6309, 6310, 6311]]


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
6302,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6304.0,NaN,NaN
6303,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6305.0,NaN,NaN
6304,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6306.0,NaN,NaN
6305,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6307.0,NaN,NaN
6306,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6308.0,NaN,NaN
6307,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6309.0,NaN,NaN
6308,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6310.0,NaN,NaN
6309,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6311,xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### No hay ninguna, ya que los valores en las columnas 'Case Number' cuando 'Date' es nulo también so nulos

### Relleno los valores nulos de la columna 'Date' 

In [17]:
df.drop(index=df[df['Date'].isna()].index, inplace=True)
df.shape

(6302, 24)

In [18]:
def clean_date(date):
    pattern = r'(\d{2})[/-](\w{3})[/-](\d{4})'  # Patrón regex para identificar la fecha
    match = re.match(pattern, date)  # Buscar el patrón en la fecha

    if match:
        day = match.group(1)  # Extraer el día
        month = match.group(2)  # Extraer el mes
        year = match.group(3)  # Extraer el año

        # Convertir el mes a formato deseado (en este caso, triple abreviatura)
        month_dict = {
            'Jan': 'Jan', 'Feb': 'Feb', 'Mar': 'Mar', 'Apr': 'Apr', 'May': 'May', 'Jun': 'Jun',
            'Jul': 'Jul', 'Aug': 'Aug', 'Sep': 'Sep', 'Oct': 'Oct', 'Nov': 'Nov', 'Dec': 'Dec'
        }

        if month in month_dict:
            month = month_dict[month]

        # Devolver la fecha en el formato deseado
        return f'{day}-{month}-{year}'

    else:
        return None  # Devolver None si no se encuentra el patrón en la fecha

In [19]:
df['Date'] = df['Date'].apply(clean_date)

df['Date'].head()

0    25-Jun-2018
1    18-Jun-2018
2    09-Jun-2018
3    08-Jun-2018
4    04-Jun-2018
Name: Date, dtype: object

In [20]:
df['Date'].isna().sum()

1549

### Cambio el tipo de dato de la columna 'Date' a datetime

In [21]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%Y')

df['Date'].head()

0   2018-06-25
1   2018-06-18
2   2018-06-09
3   2018-06-08
4   2018-06-04
Name: Date, dtype: datetime64[ns]

## 1.1 Funcion para limpiar las columnas de 'Case Numbers_' 

In [22]:
# limpio la columna con una funcion regex



def clean_date_case_number(case_number):
    case_number = str(case_number)
    pattern = r'(\d{4})\.(\d{2})\.(\d{2})'
    match = re.search(pattern, case_number)

    if match:
        year = int(match.group(1))
        month = int(match.group(2))
        day = int(match.group(3))

        if month == 0 or day == 0:
            return np.nan

        try:
            return pd.to_datetime(f'{year}-{month:02d}-{day:02d}')
        except ValueError:
            return np.nan
    else:
        return np.nan

In [23]:
df['Case Number'] = df['Case Number'].apply(clean_date_case_number)
df['Case Number'].head()

0   2018-06-25
1   2018-06-18
2   2018-06-09
3   2018-06-08
4   2018-06-04
Name: Case Number, dtype: datetime64[ns]

### Paso el tipo de dato de la columna 'Case Number' a datetime

In [24]:
df['Case Number'] = pd.to_datetime(df['Case Number'], format='%Y-%m-%d')
df['Case Number']

0      2018-06-25
1      2018-06-18
2      2018-06-09
3      2018-06-08
4      2018-06-04
          ...    
6297          NaT
6298          NaT
6299          NaT
6300          NaT
6301          NaT
Name: Case Number, Length: 6302, dtype: datetime64[ns]

In [25]:
df['Case Number'].isna().sum()

880

### Relleno los valores nulos de 'Date' con los de 'Case Number'

In [26]:
df['Date'] = df['Date'].combine_first(df['Case Number'])

df['Date']

0      2018-06-25
1      2018-06-18
2      2018-06-09
3      2018-06-08
4      2018-06-04
          ...    
6297          NaT
6298          NaT
6299          NaT
6300          NaT
6301          NaT
Name: Date, Length: 6302, dtype: datetime64[ns]

### valores nulos ahora despues de haber hecho el filtro

In [27]:
df['Date'].isna().sum()

875

In [28]:
df.drop(index=df[df['Date'].isna()].index, inplace=True)
df.shape

(5427, 24)

### Limpiar valores en 'Case Numbre.1'

In [29]:
df['Case Number.1'] = df['Case Number.1'].apply(clean_date_case_number)
df['Case Number.1'].head()

0   2018-06-25
1   2018-06-18
2   2018-06-09
3   2018-06-08
4   2018-06-04
Name: Case Number.1, dtype: datetime64[ns]

### Paso el tipo de dato de la columna 'Case Number.1' a datetime

In [30]:
df['Case Number.1'] = pd.to_datetime(df['Case Number.1'], format='%Y-%m-%d')
df['Case Number.1']

0      2018-06-25
1      2018-06-18
2      2018-06-09
3      2018-06-08
4      2018-06-04
          ...    
6150   1753-10-27
6151   1751-07-27
6155   1742-12-17
6156   1738-04-06
6160   1703-03-26
Name: Case Number.1, Length: 5427, dtype: datetime64[ns]

### Relleno los valores nulos de 'Date' con los de 'Case Number.1'

In [31]:
df['Date'] = df['Date'].combine_first(df['Case Number.1'])

df['Date']

0      2018-06-25
1      2018-06-18
2      2018-06-09
3      2018-06-08
4      2018-06-04
          ...    
6150   1753-10-27
6151   1751-07-27
6155   1742-12-17
6156   1738-04-06
6160   1703-03-26
Name: Date, Length: 5427, dtype: datetime64[ns]

### Limpiar valores en 'Case Numbre.2'

In [32]:
df['Case Number.2'] = df['Case Number.2'].apply(clean_date_case_number)
df['Case Number.2'].head()

0   2018-06-25
1   2018-06-18
2   2018-06-09
3   2018-06-08
4   2018-06-04
Name: Case Number.2, dtype: datetime64[ns]

### Paso el tipo de dato de la columna 'Case Number.2' a datetime

In [33]:
df['Case Number.2'] = pd.to_datetime(df['Case Number.2'], format='%Y-%m-%d')
df['Case Number.2']

0      2018-06-25
1      2018-06-18
2      2018-06-09
3      2018-06-08
4      2018-06-04
          ...    
6150   1753-10-27
6151   1751-07-27
6155   1742-12-17
6156   1738-04-06
6160   1703-03-26
Name: Case Number.2, Length: 5427, dtype: datetime64[ns]

### Relleno los valores nulos de 'Date' con los de 'Case Number.2'


In [34]:
df['Date'] = df['Date'].combine_first(df['Case Number.2'])

df['Date']

0      2018-06-25
1      2018-06-18
2      2018-06-09
3      2018-06-08
4      2018-06-04
          ...    
6150   1753-10-27
6151   1751-07-27
6155   1742-12-17
6156   1738-04-06
6160   1703-03-26
Name: Date, Length: 5427, dtype: datetime64[ns]

In [35]:
df['Date'].isna().sum()

0

### Reinicio el indice con las fechas ( hacer este paso al final, cuando el df esté limpio del todo)

In [36]:
df_indate = df.set_index('Date')
df_indate.head(20)

,Case Number,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
Date,,,,,,,,,,,,,,,,,,,,,,,
2018-06-25,2018-06-25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,6303.0,NaN,NaN
2018-06-18,2018-06-18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-18,2018-06-18,6302.0,NaN,NaN
2018-06-09,2018-06-09,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-09,2018-06-09,6301.0,NaN,NaN
2018-06-08,2018-06-08,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,6300.0,NaN,NaN
2018-06-04,2018-06-04,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,6299.0,NaN,NaN
2018-06-03,2018-06-03,2018.0,Unprovoked,AUSTRALIA,New South Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6298.0,NaN,NaN
2018-06-03,2018-06-03,2018.0,Unprovoked,BRAZIL,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6297.0,NaN,NaN
2018-05-27,2018-05-27,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-27,2018-05-27,6296.0,NaN,NaN
2018-05-26,2018-05-26,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,6295.0,NaN,NaN


# 2. Limpiar Type

In [37]:
df['Type'].unique()

array(['Boating', 'Unprovoked', 'Invalid', 'Provoked', 'Questionable',
       'Sea Disaster', nan, 'Boat', 'Boatomg'], dtype=object)

# 3. Limpiar Country, Location y Area

## 3.1 Country

In [38]:
df['Country'].unique()

array(['USA', 'AUSTRALIA', 'MEXICO', 'BRAZIL', 'ENGLAND', 'SOUTH AFRICA',
       'THAILAND', 'COSTA RICA', 'MALDIVES', 'BAHAMAS', 'NEW CALEDONIA',
       'ECUADOR', 'MALAYSIA', 'LIBYA', nan, 'CUBA', 'MAURITIUS',
       'NEW ZEALAND', 'SPAIN', 'SAMOA', 'SOLOMON ISLANDS', 'JAPAN',
       'EGYPT', 'ST HELENA, British overseas territory', 'COMOROS',
       'REUNION', 'FRENCH POLYNESIA', 'UNITED KINGDOM',
       'UNITED ARAB EMIRATES', 'PHILIPPINES', 'INDONESIA', 'CHINA',
       'COLUMBIA', 'CAPE VERDE', 'Fiji', 'DOMINICAN REPUBLIC',
       'CAYMAN ISLANDS', 'ARUBA', 'MOZAMBIQUE', 'PUERTO RICO', 'ITALY',
       'ATLANTIC OCEAN', 'GREECE', 'ST. MARTIN', 'FRANCE',
       'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL',
       'DIEGO GARCIA', 'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES',
       'GUAM', 'SEYCHELLES', 'BELIZE', 'NIGERIA', 'TONGA', 'SCOTLAND',
       'CANADA', 'CROATIA', 'FIJI', 'SAUDI ARABIA', 'CHILE', 'ANTIGUA',
       'KENYA', 'RUSSIA', 'TURKS & CAICOS', 'UNITE

In [39]:
def clean_strings(x):
    x = str(x)
    pattern= re.findall(r'[a-zA-Z\(\)\-\?]+', x)
    if x == 'Unknown':
        return None
    elif pattern:
        return x.lower()

In [40]:
df['Country'] = df['Country'].apply(clean_strings)
df['Country']

0             usa
1             usa
2             usa
3       australia
4          mexico
          ...    
6150      jamaica
6151          usa
6155          nan
6156        italy
6160     barbados
Name: Country, Length: 5427, dtype: object

In [41]:
df['Country'] = df['Country'].str.replace(r'[(\)\-\?]+', '')

In [42]:
df['Country'].unique()

array(['usa', 'australia', 'mexico', 'brazil', 'england', 'south africa',
       'thailand', 'costa rica', 'maldives', 'bahamas', 'new caledonia',
       'ecuador', 'malaysia', 'libya', 'nan', 'cuba', 'mauritius',
       'new zealand', 'spain', 'samoa', 'solomon islands', 'japan',
       'egypt', 'st helena, british overseas territory', 'comoros',
       'reunion', 'french polynesia', 'united kingdom',
       'united arab emirates', 'philippines', 'indonesia', 'china',
       'columbia', 'cape verde', 'fiji', 'dominican republic',
       'cayman islands', 'aruba', 'mozambique', 'puerto rico', 'italy',
       'atlantic ocean', 'greece', 'st. martin', 'france',
       'papua new guinea', 'trinidad & tobago', 'kiribati', 'israel',
       'diego garcia', 'taiwan', 'jamaica', 'palestinian territories',
       'guam', 'seychelles', 'belize', 'nigeria', 'tonga', 'scotland',
       'canada', 'croatia', 'saudi arabia', 'chile', 'antigua', 'kenya',
       'russia', 'turks & caicos', 'united arab

## 3.2 Area

In [43]:
df['Area'].unique()

array(['California', 'Georgia', 'Hawaii', 'New South Wales', 'Colima',
       'Pernambuco', 'Florida', 'Queensland', 'South Carolina',
       'Cornwall', 'Eastern Cape Province', 'Hua Hin', 'Cocos Island',
       'Western Australia', 'Alifu Alifu Atoll', 'Western Cape Province',
       'New Providence', 'Alagoas', nan, 'Victoria', 'KwaZulu-Natal',
       'Boi Island, Victoria', 'Galapagos Islands', 'Fernando de Noronha',
       'Sepang', 'Holquin Province', 'Pamplemousses ', 'South Australia',
       'North Island', 'New York', 'Canary Islands', ' Upolu Island',
       'Westerm Australia', 'Shizuoka Prefecture', 'Texas', 'Castellón',
       'Massachusetts', 'Red Sea Protectorate', 'New Providence District',
       '40 miles off Grand Bahama Island', 'Ascension Island',
       'New Jersey', 'Majorca', 'Washington', 'Tabasco', 'Anjouan',
       'Ibiza Island', 'Marquesas', 'South Devon', 'New Providence ',
       'Sharjah, ', 'Baja California Sur', 'Saint-Leu', 'South Island',
       'Lu

In [44]:
def clean_location(df, columna):
    df[columna] = df[columna].astype(str)
    df[columna] = df[columna].apply(lambda x: re.sub(r'(\d+|º)', '', x))  # eliminar datos numéricos y símbolo º
    df[columna] = df[columna].apply(lambda x: re.sub(r'\b(west|north|south|east)\b', '', x, flags=re.IGNORECASE))  # eliminar palabras "west", "north", "south" y "east"
    df[columna] = df[columna].apply(lambda x: x.strip())  # eliminar espacios en blanco al inicio o final
    df[columna] = np.where(df[columna].str.isnumeric(), np.nan, df[columna])  # valores numéricos como NaN
    return df

In [45]:
clean_location(df, 'Area')

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018-06-25,2018-06-25,2018.0,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,6303.0,NaN,NaN
1,2018-06-18,2018-06-18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-18,2018-06-18,6302.0,NaN,NaN
2,2018-06-09,2018-06-09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-09,2018-06-09,6301.0,NaN,NaN
3,2018-06-08,2018-06-08,2018.0,Unprovoked,australia,New Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,6300.0,NaN,NaN
4,2018-06-04,2018-06-04,2018.0,Provoked,mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6150,1753-10-27,1753-10-27,1753.0,Unprovoked,jamaica,nan,Kingston Harbor,Fell overboard,male,M,NaN,FATAL,Y,NaN,NaN,"Maryland Gazette, 1/31/1754",1753.10.27.R-Jamaica.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1753-10-27,1753-10-27,153.0,NaN,NaN
6151,1751-07-27,1751-07-27,1751.0,Unprovoked,usa,Massachusetts,NaN,Swimming,male,M,NaN,FATAL,Y,NaN,NaN,"Pennsylvania Gazette, 8/15/1751",1751.07.27-Massachusetts.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1751-07-27,1751-07-27,152.0,NaN,NaN
6155,1742-12-17,1742-12-17,1742.0,Unprovoked,nan,nan,Carlisle Bay,Swimming,2 impressed seamen,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",1742.12.17-AdviceSeamen.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1742-12-17,1742-12-17,148.0,NaN,NaN
6156,1738-04-06,1738-04-06,1738.0,Unprovoked,italy,Sicily,Strait of Messina,Swimming,male,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",1738.04.06.R-Messina.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1738-04-06,1738-04-06,147.0,NaN,NaN


In [46]:
valores_a_buscar = ['N, E', 'N, W', '.N .W', 'S, E']
df.loc[df['Area'].isin(valores_a_buscar), 'Area'] = np.nan

df['Area'].unique()

array(['California', 'Georgia', 'Hawaii', 'New  Wales', 'Colima',
       'Pernambuco', 'Florida', 'Queensland', 'Carolina', 'Cornwall',
       'Eastern Cape Province', 'Hua Hin', 'Cocos Island',
       'Western Australia', 'Alifu Alifu Atoll', 'Western Cape Province',
       'New Providence', 'Alagoas', 'nan', 'Victoria', 'KwaZulu-Natal',
       'Boi Island, Victoria', 'Galapagos Islands', 'Fernando de Noronha',
       'Sepang', 'Holquin Province', 'Pamplemousses', 'Australia',
       'Island', 'New York', 'Canary Islands', 'Upolu Island',
       'Westerm Australia', 'Shizuoka Prefecture', 'Texas', 'Castellón',
       'Massachusetts', 'Red Sea Protectorate', 'New Providence District',
       'miles off Grand Bahama Island', 'Ascension Island', 'New Jersey',
       'Majorca', 'Washington', 'Tabasco', 'Anjouan', 'Ibiza Island',
       'Marquesas', 'Devon', 'Sharjah,', 'Baja California Sur',
       'Saint-Leu', 'Luzon Island', 'Great Exuma', 'Saint-Andre',
       'Bimini', 'Bali', 'Tuamot

## 3.3 Location

In [47]:
df['Location'].unique()

array(['Oceanside, San Diego County', 'St. Simon Island, Glynn County',
       'Habush, Oahu', ..., 'Damiscotte', 'Carlisle Bay',
       'Strait of Messina'], dtype=object)

In [48]:
len(df['Location'].unique())

3632

In [49]:
df['Location'] = df['Location'].str.replace(r'[(\)\-\?]+', '')

# 4. Limpieza Activity e Injury

## 4.1 Activity

In [50]:
df['Activity'].unique()

array(['Paddling', 'Standing', 'Surfing', ...,
       "Standing on landed shark's tail", 'Dropped overboard',
       'Washing himself'], dtype=object)

In [51]:
len(df['Activity'].unique())

1277

In [63]:
df['Activity'] = df['Activity'].apply(clean_strings)

## 4.2 Injury

In [64]:
df['Injury'].unique()

array(['No injury to occupant, outrigger canoe and paddle damaged',
       'Minor injury to left thigh',
       'Injury to left lower leg from surfboard skeg', ...,
       'Human remains recovered from shark',
       'Many of the crew were taken by sharks',
       'Hand and foot severely bitten, surgically amputated'],
      dtype=object)

In [65]:
len(df['Injury'].unique())

3311

In [66]:
df['Activity'] = df['Activity'].apply(clean_strings)

# 5. Limpieza Name

In [52]:
df['Name'].unique()

array(['Julie Wolfe', 'Adyson\xa0McNeely ', 'John Denges', ...,
       'Joseph Feron', '2 impressed seamen',
       'Samuel Jennings, a deserter from the British frigate Milford'],
      dtype=object)

In [53]:
len(df['Name'].unique())

4655

In [54]:
df['Name'] = np.where(df['Name'].str.lower().isin(['male', 'female']), np.nan, df['Name'])
df.head(20)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018-06-25,2018-06-25,2018.0,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,6303.0,NaN,NaN
1,2018-06-18,2018-06-18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-18,2018-06-18,6302.0,NaN,NaN
2,2018-06-09,2018-06-09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-09,2018-06-09,6301.0,NaN,NaN
3,2018-06-08,2018-06-08,2018.0,Unprovoked,australia,New Wales,Arrawarra Headland,Surfing,NaN,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,6300.0,NaN,NaN
4,2018-06-04,2018-06-04,2018.0,Provoked,mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,6299.0,NaN,NaN
5,2018-06-03,2018-06-03,2018.0,Unprovoked,australia,New Wales,"Flat Rock, Ballina",Kite surfing,Chris,M,NaN,"No injury, board bitten",N,NaN,NaN,"Daily Telegraph, 6/4/2018",2018.06.03.b-FlatRock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6298.0,NaN,NaN
6,2018-06-03,2018-06-03,2018.0,Unprovoked,brazil,Pernambuco,"Piedade Beach, Recife",Swimming,Jose Ernesto da Silva,M,18,FATAL,Y,Late afternoon,Tiger shark,"Diario de Pernambuco, 6/4/2018",2018.06.03.a-daSilva.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-03,2018-06-03,6297.0,NaN,NaN
7,2018-05-27,2018-05-27,2018.0,Unprovoked,usa,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,NaN,M,52,Minor injury to foot. PROVOKED INCIDENT,N,NaN,"Lemon shark, 3'","K. McMurray, TrackingSharks.com",2018.05.27-Ponce.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-27,2018-05-27,6296.0,NaN,NaN
8,2018-05-26,2018-05-26,2018.0,Unprovoked,usa,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,M,15,Lower left leg bitten,N,17h00,"Bull shark, 6'","K.McMurray, TrackingSharks.com",2018.05.26.b-High.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,6295.0,NaN,NaN
9,2018-05-26,2018-05-26,2018.0,Unprovoked,usa,Florida,"Daytona Beach, Volusia County",Standing,NaN,M,12,Minor injury to foot,N,14h00,NaN,"K. McMurray, Tracking Sharks.com",2018.05.26.a-DaytonaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-05-26,2018-05-26,6294.0,NaN,NaN


# 8. Limpieza Sex y Age

## 8.1 Sex

In [55]:
df['Sex '].unique()

array(['F', 'M', nan, 'M ', 'lli', 'N', '.'], dtype=object)

In [56]:
def limpiar_genero(df, columna):
    df[columna] = df[columna].astype(str)
    df[columna] = df[columna].apply(lambda x: re.sub(r'[^\w\s]','',x))  # eliminar todos los símbolos
    df[columna] = df[columna].apply(lambda x: re.sub(r'\s+','',x))  # eliminar espacios en blanco
    df[columna] = np.where(df[columna].str.contains('^F|^f|^Fe|^fe', regex=True), 'F',
                           np.where(df[columna].str.contains('^M|^m|^Ma|^ma', regex=True), 'M', np.nan))
    return df
    

In [57]:
limpiar_genero(df, 'Sex ')

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018-06-25,2018-06-25,2018.0,Boating,usa,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-25,2018-06-25,6303.0,NaN,NaN
1,2018-06-18,2018-06-18,2018.0,Unprovoked,usa,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-18,2018-06-18,6302.0,NaN,NaN
2,2018-06-09,2018-06-09,2018.0,Invalid,usa,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-09,2018-06-09,6301.0,NaN,NaN
3,2018-06-08,2018-06-08,2018.0,Unprovoked,australia,New Wales,Arrawarra Headland,Surfing,NaN,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-08,2018-06-08,6300.0,NaN,NaN
4,2018-06-04,2018-06-04,2018.0,Provoked,mexico,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018-06-04,2018-06-04,6299.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6150,1753-10-27,1753-10-27,1753.0,Unprovoked,jamaica,nan,Kingston Harbor,Fell overboard,NaN,M,NaN,FATAL,Y,NaN,NaN,"Maryland Gazette, 1/31/1754",1753.10.27.R-Jamaica.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1753-10-27,1753-10-27,153.0,NaN,NaN
6151,1751-07-27,1751-07-27,1751.0,Unprovoked,usa,Massachusetts,NaN,Swimming,NaN,M,NaN,FATAL,Y,NaN,NaN,"Pennsylvania Gazette, 8/15/1751",1751.07.27-Massachusetts.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1751-07-27,1751-07-27,152.0,NaN,NaN
6155,1742-12-17,1742-12-17,1742.0,Unprovoked,nan,nan,Carlisle Bay,Swimming,2 impressed seamen,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",1742.12.17-AdviceSeamen.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1742-12-17,1742-12-17,148.0,NaN,NaN
6156,1738-04-06,1738-04-06,1738.0,Unprovoked,italy,Sicily,Strait of Messina,Swimming,NaN,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",1738.04.06.R-Messina.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1738-04-06,1738-04-06,147.0,NaN,NaN


In [58]:
df['Sex '].unique()

array(['F', 'M', 'nan'], dtype=object)

## 8.2 Age

In [59]:
df['Age'].unique()

array(['57', '11', '48', nan, '18', '52', '15', '12', '32', '10', '21',
       '30', '60', '33', '29', '54', '34', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40s', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '40', '13', '42', '3', '8',
       '50', '16', '82', '73', '20s', '68', '51', '39', '58', 'Teen',
       '47', '61', '65', '36', '66', '43', '60s', '9', '72', '59', '6',
       '27', '64', '23', '71', '44', '62', '63', '70', '18 months', '53',
       '30s', '50s', 'teen', '77', '74', '28 & 26', '5', '86', '18 or 20',
       '12 or 13', '46 & 34', '28, 23 & 30', 'Teens', '36 & 26',
       '8 or 10', '84', '\xa0 ', ' ', '30 or 36', '6½', '21 & ?', '75',
       '33 or 37', 'mid-30s', '23 & 20', ' 30', '7      &    31', ' 28',
       '20?', '32 & 30', '16 to 18', '87', '67', 'Elderly', 'mid-20s',
       '74 ', '45 ', '21 or 26', '20 ', '>50', '18 to 22', 'adult',
       '9 & 12', '? & 19', '(adult)', '33 & 37', '25 or 28',
       '37

In [60]:
def age_filt(x):
    x=str(x)
    age_pattern = r"\b(\d{2})\b"  # Extracts two-digit numbers
    match = re.search(age_pattern, x)
    
    if match:
        age = int(match.group(1))
        return int(age)
    else:
        return np.nan

In [61]:
df['Age'] = df['Age'].apply(age_filt)
df['Age']

0       57.0
1       11.0
2       48.0
3        NaN
4        NaN
        ... 
6150     NaN
6151     NaN
6155     NaN
6156     NaN
6160    19.0
Name: Age, Length: 5427, dtype: float64

In [62]:
df['Age'].unique()

array([57., 11., 48., nan, 18., 52., 15., 12., 32., 10., 21., 30., 60.,
       33., 29., 54., 34., 41., 37., 56., 19., 25., 69., 38., 55., 35.,
       46., 45., 14., 28., 20., 24., 26., 49., 22., 31., 17., 40., 13.,
       42., 50., 16., 82., 73., 68., 51., 39., 58., 47., 61., 65., 36.,
       66., 43., 72., 59., 27., 64., 23., 71., 44., 62., 63., 70., 53.,
       77., 74., 86., 84., 75., 87., 67., 81., 78.])